# Nationalratswahlen

In [1]:
!pip install pyaxis

     |████████████████████████████████| 15.7 MB 8.3 MB/s eta 0:00:01
     |████████████████████████████████| 11.5 MB 47.6 MB/s eta 0:00:01
     |████████████████████████████████| 794 kB 45.4 MB/s eta 0:00:01
  Created wheel for pyaxis: filename=pyaxis-0.3.4-py3-none-any.whl size=10766 sha256=50fac3a602c23396fb67b314571737af26de5f00be1a9a8d7b1b047441d606b0
  Stored in directory: /home/jovyan/.cache/pip/wheels/62/9d/72/93ff83bda641a1843128911c9a94a7f739aaf4788388f5db67
  Created wheel for pyjstat: filename=pyjstat-2.2.1-py3-none-any.whl size=19429 sha256=940ccd705bb3ea8c29d309b1d625e002e9c52848c8a08520b700b2fe54b1bed3
  Stored in directory: /home/jovyan/.cache/pip/wheels/94/28/15/0c98e3d81093d35fb6bd1f28295e145788970d18b7a0076f22
Successfully built pyaxis pyjstat
You should consider upgrading via the '/opt/conda/bin/python3 -m pip install --upgrade pip' command.


In [4]:
#!/usr/bin/python
# -*- coding: utf-8 -*-

In [2]:
from pyaxis import pyaxis
import pandas as pd
import sys

In [17]:
from constants import INDICATOR_ID_MAPPING, URL_MAPPING, CSV_MAPPING, METADATA_MAPPING
from utils import map_name_to_number

In [3]:
# load data from bfs
URL = 'https://www.bfs.admin.ch/bfsstatic/dam/assets/12948000/master'

In [4]:
px = pyaxis.parse(URL, encoding='ISO-8859-2')

In [5]:
df = px['DATA']
print(df.shape)
df.head()

(570024, 4)


,Kanton (-) / Bezirk (>>) / Gemeinde (......),Jahr,Ergebnisse,DATA
0,Schweiz,2019,Wahlberechtigte,5459218
1,Schweiz,2019,Wählende / Eingelegte Wahlzettel (WZ),2462641
2,Schweiz,2019,Wahlbeteiligung [%],45.1097758
3,Schweiz,2019,Leere WZ,9366
4,Schweiz,2019,Ungültige WZ,29015


In [6]:
# rename 'Kanton (-) / Bezirk (>>) / Gemeinde (......)' to 'name_de'
df = df.rename(columns={
    'Kanton (-) / Bezirk (>>) / Gemeinde (......)': 'name_de', 
    'Jahr': 'year', 
    'Ergebnisse': 'results', 
    'DATA': 'data'
})
df.head()

,name_de,year,results,data
0,Schweiz,2019,Wahlberechtigte,5459218
1,Schweiz,2019,Wählende / Eingelegte Wahlzettel (WZ),2462641
2,Schweiz,2019,Wahlbeteiligung [%],45.1097758
3,Schweiz,2019,Leere WZ,9366
4,Schweiz,2019,Ungültige WZ,29015


In [7]:
df['name_de'].unique()

array(['Schweiz', 'Zürich', '>> Bezirk Affoltern', ...,
       '......Clos du Doubs', '......Haute-Ajoie', '......La Baroche'],
      dtype=object)

In [8]:
df['name_de'] = df['name_de'].str.replace('.', '')
df['name_de'] = df['name_de'].str.replace('>> ', '')
df['name_de'] = df['name_de'].str.replace('   ', '')

/tmp/ipykernel_493/2170717606.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['name_de'] = df['name_de'].str.replace('.', '')


In [9]:
set(df['results'])

{'Gültige WZ',
 'Kandidatenstimmen aus unveränderten WZ',
 'Kandidatenstimmen aus veränderten WZ',
 'Leere WZ',
 'Listenstimmen aus unveränderten WZ',
 'Listenstimmen aus veränderten WZ',
 'Total Kandidatenstimmen',
 'Total Listenstimmen',
 'Total Zusatzstimmen',
 'Ungültige WZ',
 'Unveränderte WZ',
 'Veränderte WZ',
 'WZ ohne Listenbezeichnung',
 'Wahlberechtigte',
 'Wahlbeteiligung [%]',
 'Wählende / Eingelegte Wahlzettel (WZ)',
 'Zusatzstimmen aus unveränderten WZ',
 'Zusatzstimmen aus veränderten WZ'}

In [33]:
metadata= {}
for col in ['name_de', 'results']:
    conversion_dict = {}
    options = set(df[col])
    for i, option in enumerate(options):
        conversion_dict[option] = i
    df[col] = df[col].map(conversion_dict)
    metadata[col] = conversion_dict

In [11]:
def map_name_to_number(df, columns_to_map):
    metadata= {}
    for col in columns_to_map:
        conversion_dict = {}
        options = set(df[col])
        for i, option in enumerate(options):
            conversion_dict[option] = i
        df[col] = df[col].map(conversion_dict)
        metadata[col] = conversion_dict
    return df, metadata

In [13]:
df, metadata = map_name_to_number(df, columns_to_map=['name_de', 'results'])

In [14]:
df.head()

,name_de,year,results,data
0,1811,2019,8,5459218
1,1811,2019,9,2462641
2,1811,2019,0,45.1097758
3,1811,2019,16,9366
4,1811,2019,10,29015


In [15]:
metadata

{'name_de': {'Donat': 0,
  'Le Landeron': 1,
  'Mörel-Filet': 2,
  'Medel (Lucmagn)': 3,
  'Hellsau (Gemeinsame Urne, Standort: Höchstetten )': 4,
  'Vorderthal': 5,
  'Montagny (FR)': 6,
  'Rüschlikon': 7,
  'Brienzwiler': 8,
  'Zofingen': 9,
  'Freienbach': 10,
  'Abtwil': 11,
  "L'Abbaye": 12,
  'Vendlincourt': 13,
  'Endingen': 14,
  'Buchs (AG)': 15,
  'Trélex': 16,
  'Brügg': 17,
  'Gaiserwald': 18,
  'Montagny-prčs-Yverdon': 19,
  'Rovray': 20,
  'Calanca': 21,
  'Härkingen': 22,
  'Schwadernau': 23,
  'Niederglatt': 24,
  'Warth-Weiningen': 25,
  'Chavannes-de-Bogis': 26,
  'Bioley-Magnoux': 27,
  'Moiry': 28,
  'Hauenstein-Ifenthal': 29,
  'Hausen (AG)': 30,
  'Schüpfheim': 31,
  'Bagnes': 32,
  'Bezirk Dietikon': 33,
  'Huttwil': 34,
  'Entlebuch': 35,
  'Muotathal': 36,
  'Höri': 37,
  'Baulmes': 38,
  'Biberist': 39,
  'Alpthal': 40,
  'Unterengstringen': 41,
  'Rueyres': 42,
  'Embrach': 43,
  'La Folliaz': 44,
  'Mont-Vully': 45,
  'Epsach': 46,
  'Neunkirch': 47,
  'Bell

In [39]:
# create new dataframe
df = pd.DataFrame({
    'indicator_id': 'TBD',
    'spatialunit_id': df['name_de'],
    'time_value': df['year'],
    'time_info_id': 1,
    'results': df['results'],
    'value': df['data']
})

In [40]:
df.head()

,indicator_id,spatialunit_id,time_value,time_info_id,results,value
0,TBD,459,2019,1,11,5459218
1,TBD,459,2019,1,13,2462641
2,TBD,459,2019,1,7,45.1097758
3,TBD,459,2019,1,8,9366
4,TBD,459,2019,1,14,29015


In [19]:
# export as csv
df.to_csv('national_council_elections.csv', index=False, na_rep='NA')